# Approximate 1D by FDM

In [1]:
import numpy, math

#Problem function F(u)=f(x)
def f(x):
    return 1-x

#Boundary Conditions
def bf(x):
    return 3-4*x


def Uf(number_intervals,a,b):
    h = (b-a)/number_intervals
    X = [a+i*h for i in range(number_intervals+1)]

    #approximated function (c_i-1)(u_i-1)+(c_i)(u_i)+(c_i+1)(u_i+1)=f(x)
    #Enter ONLY c_i-1,c_i,c_i+1
    C = [-1/h/h, 2/h/h+1, -1/h/h]

    Ai = (number_intervals-1)*[0]
    Ai[:2] = C[1:]
    A = [Ai]
    B = [[f(X[1])-C[0]*bf(a)]]
    for i in range(1,number_intervals-2):
        Ai = (number_intervals-1)*[0]
        Ai[i-1:i+2] = C
        A.append(Ai)
        B.append([f(X[i+1])])
    Ai = (number_intervals-1)*[0]
    Ai[-2:] = C[:2]
    A.append(Ai)
    B.append([f(X[number_intervals-1])-C[2]*bf(b)])

    A = numpy.array(A)
    B = numpy.array(B)
    return numpy.linalg.inv(A).dot(B)

print(Uf(5,0,1))

[[ 2.14027673]
 [ 1.33416453]
 [ 0.55741892]
 [-0.21302994]]


# Compare to exact solution (1D)

In [2]:
#exact solution
e = math.e
def ef(x):
    c1 = (2+e)/(1-e*e)
    c2 = 2-(2+e)/(1-e*e)
    return c1*math.pow(e,x)+c2*math.pow(e,-x)-x+1

def compare(number_intervals,a,b):
    h = (b-a)/number_intervals
    U = Uf(number_intervals,a,b)
    
    #exact solution points E
    E = []
    for i in range(1,number_intervals):
        E.append([ef(a+i*h)])

    #measure by RMSE(L2-norm)
    RMSE = numpy.sqrt(numpy.square(numpy.subtract(E,U)).mean())
    return h,RMSE

for i in range(3):
    h,RMSE = compare(int(math.pow(10,i+1)/2),0,1)
    print('h =',h,'\t RMSE =',RMSE,'\n')
    h,RMSE = compare(int(math.pow(10,i+1)),0,1)
    print('h =',h,'\t RMSE =',RMSE,'\n')


h = 0.2 	 RMSE = 0.0001517386539341629 

h = 0.1 	 RMSE = 3.6021009621095275e-05 

h = 0.02 	 RMSE = 1.3827733024229342e-06 

h = 0.01 	 RMSE = 3.439561409467372e-07 

h = 0.002 	 RMSE = 1.370320445046174e-08 

h = 0.001 	 RMSE = 3.423994660686047e-09 



## Approximate 2D by FDM

In [8]:
import numpy
import math

#Problem function F(u)=f(x,y)
def f(x,y):
    return 4*(math.pi)*(math.pi)*math.sin(2*math.pi*x)*y*y*(1-y)*(1-y)-math.sin(2*math.pi*x)*(2-12*y+12*y*y)

#Boundary Conditions
def bf(x,y):
    return 0

import numpy
def Uf(nix,ax,bx,niy,ay,by):
    hx = (bx-ax)/nix
    hy = (by-ay)/niy
    X = [[ax+i*hx for i in range(nix+1)],[ay+j*hy for j in range(niy+1)]]

    '''approximated coefficient 
    [[(c_i-1,j-1),(c_i-1,j),(c_i-1,j+1)]
     [ (c_i,j-1) , (c_i,j) , (c_i,j+1) ]
     [(c_i+1,j-1),(c_i+1,j),(c_i+1,j+1)]]
    
    '''
    
    hx2 = hx*hx
    hy2 = hy*hy
    C = [[   0   , -1/hx2     ,   0   ],
         [ -1/hy2, 2/hx2+2/hy2, -1/hy2],
         [   0   , -1/hx2     ,   0   ]]
    
    sx = 2*nix-3
    sx = 3 if nix==1 else sx
    sy = 2*niy-3
    sy = 3 if niy==1 else sy
    
    SC = numpy.zeros((sx,sy))
    SC[int(sx/2)-1:int(sx/2)+2:1, int(sy/2)-1:int(sy/2)+2:1] = C
    A = []
    B = []
    for i in range(1,nix):
        for j in range(1,niy):
            Ai = SC[nix-i-1:2*nix-i-2:1, niy-j-1:2*niy-j-2:1]
            A.append(Ai)            
    A = numpy.array(A).reshape((nix-1)*(niy-1),(nix-1)*(niy-1))
    
    SB = numpy.zeros((nix+1,niy+1))
    for i in range(nix+1):
        for j in range(niy+1):
            if i==0 or j==0 or i==nix or j==niy:
                SB[i][j]=bf(X[0][i],X[1][j])
    B = []
    for i in range(nix-1):
        for j in range(niy-1):
            Bi = []
            Bi.append(f(X[0][i+1],X[1][j+1])-numpy.sum(C*SB[i:i+3:1,j:j+3:1]))
            B.append(Bi)
    B = numpy.array(B)
    return numpy.linalg.inv(A).dot(B)

Uf(4,0,1,3,0,1)

array([[ 6.38101989e-02],
       [ 6.38101989e-02],
       [ 1.21430643e-17],
       [ 1.04083409e-17],
       [-6.38101989e-02],
       [-6.38101989e-02]])

## Compare to exact solution (2D)

In [4]:
#exact solution
e = math.e
def ef(x,y):
    return math.sin(2*math.pi*x)*y*y*(1-y)*(1-y)

def compare(nix,ax,bx,niy,ay,by):
    hx = (bx-ax)/nix
    hy = (by-ay)/niy
    U = Uf(nix,ax,bx,niy,ay,by)
    
    #exact solution points E
    E = []
    for i in range(1,nix):
        for j in range(1,niy):
            E.append([ef(ax+i*hx,ay+j*hy)])

    #measure by RMSE(L2-norm)
    RMSE = numpy.sqrt(numpy.square(numpy.subtract(E,U)).mean())
    return hx,hy,RMSE
    
for i in range(2,6):
    hx,hy,RMSE = compare(int(math.pow(2,i)),0,1,int(math.pow(3,i-1)),0,1)
    print('hx =',round(hx,6),'\t hy =',round(hy,6),'\t RMSE =',RMSE,'\n')

hx = 0.25 	 hy = 0.333333 	 RMSE = 0.011779990387091023 

hx = 0.125 	 hy = 0.111111 	 RMSE = 0.001702260814235706 

hx = 0.0625 	 hy = 0.037037 	 RMSE = 0.00034076132264646366 

hx = 0.03125 	 hy = 0.012346 	 RMSE = 7.738322685873063e-05 



# END

In [5]:
A = numpy.arange(9).reshape(3,3)
nix = 3
niy = 4
sx = 2*nix-3
sx = 3 if nix==1 else sx
sy = 2*niy-3
sy = 3 if niy==1 else sy
    
SC = numpy.zeros((sx,sy))
SC[int(sx/2)-1:int(sx/2)+2:1, int(sy/2)-1:int(sy/2)+2:1] = A
SC

array([[0., 0., 1., 2., 0.],
       [0., 3., 4., 5., 0.],
       [0., 6., 7., 8., 0.]])

In [6]:
A = numpy.arange(4).reshape(2,2)
B = numpy.arange(1,5).reshape(2,2)
numpy.sum(A*B)

20

# problem 6

In [7]:


def u(x,y):
    return math.sin(2*math.pi*x)*y*y*(1-y)*(1-y)

A = numpy.array([[50,-9,-16,0,0,0], [-9,50,0,-16,0,0], [-16,0,50,-9,-16,0], [0,-16,-9,50,0,-16], [0,0,-16,0,50,-9], [0,0,0,-16,-9,50]])
B = numpy.array([[f(1/4,1/3)],[f(1/4,2/3)],[f(2/4,1/3)],[f(2/4,2/3)],[f(3/4,1/3)],[f(3/4,2/3)]])
X = numpy.linalg.inv(A).dot(B)

print(B)

print([[u(1/4,1/3)],[u(1/4,2/3)],[u(2/4,1/3)],[u(2/4,2/3)],[u(3/4,1/3)],[u(3/4,2/3)]])

[[ 2.61621815e+00]
 [ 2.61621815e+00]
 [ 3.20394319e-16]
 [ 3.20394319e-16]
 [-2.61621815e+00]
 [-2.61621815e+00]]
[[0.04938271604938273], [0.04938271604938273], [6.047638514307918e-18], [6.047638514307918e-18], [-0.04938271604938273], [-0.04938271604938273]]
